In [1]:
print("OK")

OK


In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Ganesh\\AIML Engineer\\DL (End-to-End Project)\\End-to-End-Chest-Cancer-Classification-using-MLFlow-DVC-Deep-Learning-Project-\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Ganesh\\AIML Engineer\\DL (End-to-End Project)\\End-to-End-Chest-Cancer-Classification-using-MLFlow-DVC-Deep-Learning-Project-'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

[2025-12-03 12:00:23,317: WARNING: module_wrapper: From c:\Users\Ganesh\anaconda3\envs\cancer\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [10]:
import tensorflow as tf
from pathlib import Path

class PrepareBaseModel:
    def __init__(self, config):
        """
        Initializes the PrepareBaseModel with a configuration object.

        Args:
            config: PrepareBaseModelConfig object containing paths and parameters
        """
        self.config = config
        self.model = None
        self.full_model = None

    def get_base_model(self):
        """
        Loads the base VGG16 model with specified configuration and saves it.
        """
        self.model = tf.keras.applications.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        # Save the base model
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        """
        Prepares the full model with custom classification head.

        Args:
            model: Base model (e.g., VGG16)
            classes: Number of output classes
            freeze_all: If True, freeze all layers
            freeze_till: Number of layers to keep trainable from the end
            learning_rate: Learning rate for optimizer

        Returns:
            Compiled tf.keras Model
        """
        # Freeze layers as required
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif freeze_till is not None and freeze_till > 0:
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        # Add custom classification head
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        # Compile the model
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        """
        Updates the base model by adding a custom classification head
        and freezes layers as configured.
        """
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        # Save the updated base model
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """
        Saves the given model to the specified path.

        Args:
            path: Path to save the model
            model: tf.keras Model instance
        """
        path.parent.mkdir(parents=True, exist_ok=True)  # Ensure folder exists
        model.save(path)


In [11]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-12-03 12:00:43,516: INFO: common: yaml file: config\config.yaml loaded successfully]


[2025-12-03 12:00:43,544: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-03 12:00:43,550: INFO: common: created directory at: artifacts]
[2025-12-03 12:00:43,556: INFO: common: created directory at: artifacts/prepare_base_model]
[2025-12-03 12:00:43,897: WARNING: module_wrapper: From c:\Users\Ganesh\anaconda3\envs\cancer\lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
[2025-12-03 12:00:44,154: WARNING: module_wrapper: From c:\Users\Ganesh\anaconda3\envs\cancer\lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.
]
58889256/58889256 [==============================] - 6s 0us/step
[2025-12-03 12:00:51,648: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train

c:\Users\Ganesh\anaconda3\envs\cancer\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     